# Introduction

QA plots for the INTT detector

In [ ]:
# imports to write dynamic markdown contents
import os
from IPython.display import display, Markdown, Latex
from IPython.display import HTML

In [ ]:
# turn off/on code for the result HTML page

display(Markdown('*For the result HTML page:* '))
    
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [ ]:
import os.path

# readme file of the macros, available if run under JenkinsCI
# https://github.com/sPHENIX-Collaboration/utilities/blob/master/jenkins/built-test/test-tracking-qa.sh 

macro_markdown = 'Fun4All-macros-README.md'

if os.path.isfile(macro_markdown) :
    with open(macro_markdown, 'r') as file:
        display(Markdown(file.read()))

## `pyROOT` env check

In [ ]:
import ROOT

OFFLINE_MAIN = os.getenv("OFFLINE_MAIN")
if OFFLINE_MAIN is not None:
    display(Markdown(f"via sPHENIX software distribution at `{OFFLINE_MAIN}`"))

## Plotting source code

In [ ]:
import subprocess

try:
    git_url = \
        subprocess.run(['git','remote','get-url','origin'], stdout=subprocess.PIPE)\
        .stdout.decode('utf-8').strip()\
        .replace('git@github.com:','https://github.com/')

    display(Markdown(f"View the source code repository at {git_url}"))
except: # catch *all* exceptions
    #     well do nothing
    pass

## JenkinsCI information (if available)

In [ ]:
display(Markdown('Some further details about the QA run, if executed under the Jenkins CI:'))

checkrun_repo_commit = os.getenv("checkrun_repo_commit")
if checkrun_repo_commit is not None:
    display(Markdown(f"* The commit being checked is {checkrun_repo_commit}"))
        
ghprbPullLink = os.getenv("ghprbPullLink")
if ghprbPullLink is not None:
    display(Markdown(f"* Link to the pull request: {ghprbPullLink}"))

BUILD_URL =  os.getenv("BUILD_URL")
if BUILD_URL is not None:
    display(Markdown(f"* Link to the build: {BUILD_URL}"))

git_url_macros =  os.getenv("git_url_macros")
sha_macros =  os.getenv("sha_macros")
if git_url_macros is not None:
    display(Markdown(f"* Git repo for macros: {git_url_macros} , which merges `{sha_macros}` and the QA tracking branch"))

RUN_ARTIFACTS_DISPLAY_URL = os.getenv("RUN_ARTIFACTS_DISPLAY_URL")
if RUN_ARTIFACTS_DISPLAY_URL is not None:
    display(Markdown(f"* Download the QA ROOT files: {RUN_ARTIFACTS_DISPLAY_URL}"))

JENKINS_URL  = os.getenv("JENKINS_URL")
if JENKINS_URL is not None:
    display(Markdown(f"Automatically generated by [sPHENIX Jenkins continuous integration]({JENKINS_URL}) [![sPHENIX](https://raw.githubusercontent.com/sPHENIX-Collaboration/utilities/master/jenkins/material/sphenix-logo-white-bg-72p.png)](https://www.sphenix.bnl.gov/web/) &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; [![jenkins.io](https://raw.githubusercontent.com/sPHENIX-Collaboration/utilities/master/jenkins/material/jenkins_logo_title-72p.png)](https://jenkins.io/)"))


# Initialization

In [ ]:
%%cpp -d

#include "QA_Draw_Utility.C"

#include <sPhenixStyle.C>

#include <TFile.h>
#include <TLine.h>
#include <TString.h>
#include <TTree.h>
#include <cassert>
#include <cmath>

In [ ]:
%%cpp

SetsPhenixStyle();
TVirtualFitter::SetDefaultFitter("Minuit2");

// test sPHENIX lib load
// gSystem->Load("libg4eval.so");

// test libs
// gSystem->ListLibraries();

## Inputs and file checks

In [ ]:

qa_file_name_new = os.getenv("qa_file_name_new")
if qa_file_name_new is None:
    qa_file_name_new = "G4sPHENIX_test-tracking-low-occupancy-qa_Event100_Sum10_qa.root"
    display(Markdown(f"`qa_file_name_new` env not set. use the default `qa_file_name_new={qa_file_name_new}`"))

qa_file_name_ref = os.getenv("qa_file_name_ref")
if qa_file_name_ref is None:
    qa_file_name_ref = "reference/G4sPHENIX_test-tracking-low-occupancy-qa_Event100_Sum10_qa.root"
    display(Markdown(f"`qa_file_name_ref` env not set. use the default `qa_file_name_ref={qa_file_name_ref}`"))
elif qa_file_name_ref == 'None':
    qa_file_name_ref = None
    display(Markdown(f"`qa_file_name_ref` = None and we are set to not to use the reference histograms"))
    


In [ ]:
# qa_file_new = ROOT.TFile.Open(qa_file_name_new);

# assert qa_file_new.IsOpen()
# qa_file_new.ls()
display(Markdown(f"Openning QA file at `{qa_file_name_new}`"))
ROOT.gInterpreter.ProcessLine(f"TFile *qa_file_new = new TFile(\"{qa_file_name_new}\");")
ROOT.gInterpreter.ProcessLine(f"const char * qa_file_name_new = \"{qa_file_name_new}\";")

if qa_file_name_ref is not None:
#     qa_file_ref = ROOT.TFile.Open(qa_file_name_ref);

#     assert qa_file_ref.IsOpen()
    display(Markdown(f"Openning QA reference file at `{qa_file_name_ref}`"))
    ROOT.gInterpreter.ProcessLine(f"TFile *qa_file_ref = new TFile(\"{qa_file_name_ref}\");")
    ROOT.gInterpreter.ProcessLine(f"const char *  qa_file_name_ref = \"{qa_file_name_ref}\";")
else:
    ROOT.gInterpreter.ProcessLine(f"TFile *qa_file_ref = nullptr;")
    ROOT.gInterpreter.ProcessLine(f"const char *  qa_file_name_ref = nullptr;")


In [ ]:
%%cpp

if (qa_file_new == nullptr) 
{
    cout <<"Error, can not open QA root file"<<qa_file_name_new<<endl;
    exit(1);
}

// list inputs histograms if needed
// qa_file_new ->ls();

//TFile *qa_file_ref = NULL;
//if (qa_file_name_ref)
//{
//    qa_file_ref = new TFile(qa_file_name_ref);
//  
//    if (qa_file_ref == nullptr) 
//    {
//        cout <<"Error, can not open QA root file"<<qa_file_name_ref<<endl;
//        exit(1);
//    }
//}

## Utilities (invisible to HTML output)

In [ ]:
%%cpp
//constants

const char *hist_name_prefix = "QAG4SimulationIntt";

// assume INTT layers are 3 to 6
static constexpr int first_layer_intt = 3;
static constexpr int nlayers_intt = 4;

In [ ]:
%%cpp -d
// utility functions for batch plotting

TCanvas* Draw( TFile* qa_file_new, TFile* qa_file_ref, const TString& hist_name_prefix, const TString& tag )
{

    const TString prefix = TString("h_") + hist_name_prefix + TString("_");

    auto cv = new TCanvas(
      TString("QA_Draw_Intt_") + tag + TString("_") + hist_name_prefix,
      TString("QA_Draw_Intt_") + tag + TString("_") + hist_name_prefix,
      1800, 1000);

    DivideCanvas( cv, nlayers_intt );
    for( int ilayer = 0; ilayer < nlayers_intt; ++ilayer )
    {

      const int layer = ilayer + first_layer_intt;

      // get histograms
      auto hnew = static_cast<TH1*>( qa_file_new->GetObjectChecked( Form( "%s%s_%i", prefix.Data(), tag.Data(), layer ), "TH1" ) );
      hnew->Scale( 1./hnew->GetEntries() );
      hnew->SetMinimum(0);

      // reference
      auto href = qa_file_ref ? static_cast<TH1*>( qa_file_ref->GetObjectChecked( Form( "%s%s_%i", prefix.Data(), tag.Data(), layer ), "TH1" ) ) : nullptr;
      if( href )
      {
        href->Scale( 1./href->GetEntries() );
        href->SetMinimum(0);
      }

      // draw
      cv->cd( ilayer+1 );
      DrawReference(hnew, href);

      auto line = VerticalLine( gPad, 0 );
      line->Draw();
    }

    return cv;

}


# Azimuthal arc-length, $r \Delta\phi$, residuals, errors and pulls

## Residuals

In [ ]:
%%cpp

Draw( qa_file_new, qa_file_ref, hist_name_prefix, "drphi" )-> Draw();

## Errors 

In [ ]:
%%cpp

Draw( qa_file_new, qa_file_ref, hist_name_prefix, "rphi_error" )-> Draw();

## Pulls

In [ ]:
%%cpp

Draw( qa_file_new, qa_file_ref, hist_name_prefix, "phi_pulls" )-> Draw();

# $z$ residuals, errors and pulls

## Residuals

In [ ]:
%%cpp

Draw( qa_file_new, qa_file_ref, hist_name_prefix, "dz" )-> Draw();

## Errors 

In [ ]:
%%cpp

Draw( qa_file_new, qa_file_ref, hist_name_prefix, "z_error" )-> Draw();

## Pulls

In [ ]:
%%cpp

Draw( qa_file_new, qa_file_ref, hist_name_prefix, "z_pulls" )-> Draw();

# Cluster sizes

## Cluster size

In [ ]:
%%cpp

Draw( qa_file_new, qa_file_ref, hist_name_prefix, "clus_size" )-> Draw();

## Cluster azimuthal size 

In [ ]:
%%cpp

Draw( qa_file_new, qa_file_ref, hist_name_prefix, "clus_size_phi" )-> Draw();

## Cluster $z$-size

In [ ]:
%%cpp

Draw( qa_file_new, qa_file_ref, hist_name_prefix, "clus_size_z" )-> Draw();

# Summary statistics

In [ ]:
%%cpp

KSTestSummary::getInstance()->make_summary_txt("QA-Intt.txt");

In [ ]:
%%cpp

KSTestSummary::getInstance()->make_summary_TCanvas() -> Draw();